In [1]:
from googleads import dfp
dfp_client = dfp.DfpClient.LoadFromStorage()
network_service = dfp_client.GetService('NetworkService', version='v201705')
current_network = network_service.getCurrentNetwork()
print 'Found network %s (%s)!' % (current_network['displayName'],current_network['networkCode'])

Found network BlackBerry Limited (90363076)!


In [8]:
from datetime import datetime
from datetime import timedelta
import tempfile

# Import appropriate modules from the client library.
from googleads import dfp
from googleads import errors

ORDER_ID = '2129581187'


def main(client, order_id):
  # Create statement object to filter for an order.
  values = [{
      'key': 'id',
      'value': {
          'xsi_type': 'NumberValue',
          'value': order_id
      }
  }]
  filter_statement = {'query': 'WHERE ORDER_ID = :id',
                      'values': values}

  # Set the start and end dates of the report to run (past 8 days).
  end_date = datetime.now().date()
  start_date = end_date - timedelta(days=8)

  # Create report job.
  report_job = {
      'reportQuery': {
          'dimensions': ['ORDER_ID','ORDER_NAME','LINE_ITEM_ID','LINE_ITEM_NAME','LINE_ITEM_TYPE','CREATIVE_ID', 'CREATIVE_NAME','CREATIVE_TYPE','NATIVE_TEMPLATE_ID','NATIVE_TEMPLATE_NAME'],
          'dimensionAttributes': ['CREATIVE_CLICK_THROUGH_URL'],
          'statement': filter_statement,
          'columns': ['AD_SERVER_IMPRESSIONS', 'AD_SERVER_CLICKS',
                      'AD_SERVER_CTR', 'AD_SERVER_CPM_AND_CPC_REVENUE',
                      'AD_SERVER_WITHOUT_CPD_AVERAGE_ECPM'],
          'dateRangeType': 'CUSTOM_DATE',
          'startDate': start_date,
          'endDate': end_date
      }
  }

  # Initialize a DataDownloader.
  report_downloader = client.GetDataDownloader(version='v201708')

  try:
    # Run the report and wait for it to finish.
    report_job_id = report_downloader.WaitForReport(report_job)
  except errors.DfpReportError, e:
    print 'Failed to generate report. Error was: %s' % e

  # Change to your preferred export format.
  export_format = 'CSV_DUMP'

  report_file = open('/root/notebooks/query_test.csv.gz','w')

  # Download report data.
  report_downloader.DownloadReportToFile(
      report_job_id, export_format, report_file)

  report_file.close()

  # Display results.
  print 'Report job with id "%s" downloaded to:\n%s' % (
      report_job_id, report_file.name)

if __name__ == '__main__':
  # Initialize client object.
  dfp_client = dfp.DfpClient.LoadFromStorage()
  main(dfp_client, ORDER_ID)



Report job with id "10027488633" downloaded to:
/root/notebooks/query_test.csv.gz


NameError: name 'report_file' is not defined

In [11]:
import csv

with open('/root/notebooks/query_test.csv.gz', 'rb') as report:
    report_reader = csv.reader(report)
    for row in report_reader:
      print row

Error: line contains NULL byte